<a href="https://colab.research.google.com/github/ovomartina/CSF-dynamics_models/blob/main/Chu_et_al_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Choose test type: 'THCT', 'infusion', or 'ABP_drop'
test = 'THCT'

# Global list to store CPP history
cpp_history = []
ABP_history = []

# Define time parameters
total_time = 200 # 2 * 60  # 2 minutes in seconds
step_duration = 1 * 60  # 1 minutes in seconds per step

# Parameters (example values, adjust as needed)
Ra =  0.1 # [mmHg·min/mL] Arterial resistance
CVR_max = 0.5 # [mmHg·min/mL]
CVR_min = 0.1 # [mmHg·min/mL]
CPP_max = 95 # [mmHg]
CPP_min = 55 # [mmHg]
CPP_offset = -30 #[mmHg]
Caspan = 0.01 # [mL/mmHg]
Coffset = 0.05 # [mL/mmHg]
Cv = 1.0    # [mL/mmHg] Compliance of vein
Rb0 = 0.7    # [mmHg·min/mL] Resistance between vein and CSF
E = 0.05*30 # [1/mL] Pathological value
P0 = 5. # [mmHg]
P_breakpoint = 10. # [mmHg]
If = 0.3    # [mL/min]

# Values not mentioned in the paper
Pss = 7.4 # [mmHg] from https://pmc.ncbi.nlm.nih.gov/articles/PMC8447530/pdf/12987_2021_Article_274.pdf
Risf_span = 20 # [mmHg·min/mL] from https://pmc.ncbi.nlm.nih.gov/articles/PMC8447530/pdf/12987_2021_Article_274.pdf
Risf_offset = 0
RCSF = 16*30 # Pathological value of CSF drainage resistance [mmHg·min/mL]
# Initial conditions
Pa0 = 120.0  # Initial arterial pressure
Pv0 = 12.0  # Initial venous pressure
Pi0 = 10.0  # Initial intracranial pressure
y0 = [Pa0, Pv0, Pi0]

def CVR_m(CPP_m):
  return CVR_max-(CVR_max-CVR_min)/(1+((CPP_m+CPP_offset)/(CPP_max-CPP_min))**6)

# Define the ODE system
def csf_dynamics(t, y):
    # Unpacking variables
    Pa, Pv, Pi = y

    Rb = 10_000 if Pv <= Pi else Rb0 / (Pv - Pi)

    Ci = 1 / (E * (Pi - P0)) if Pi >= P_breakpoint else 1 / (E * (P_breakpoint - P0)) # Intracranial compliance

    I_inf=0
    # Select ABP based on test scenario
    I_inf = 0
    if test == 'THCT':
        ABP_mean = 90 if 13 <= t < 23 else 120 if t < 12 or t >= 24 else 120 - 30 * (t - 12) if t < 13 else 90 + 30 * (t - 23)
        ABP = ABP_mean + 20 * np.sin(2 * np.pi * t / 1)
    elif test == 'infusion':
        ABP_mean = 120
        I_inf = 5.0 if 30 < t < 80 else 0.0
        ABP = ABP_mean + 20 * np.sin(2 * np.pi * t / 1)
    elif test == 'ABP_drop':
        ABP = 70 if t > 60 else 120

    ABP_history.append((t, ABP))

    # Compute CPP (Cerebral Perfusion Pressure)
    CPP = ABP - Pi
    cpp_history.append((t, CPP))

    # Compute CPP_m as the mean of last 6s
    past_cpp = [cpp for time, cpp in cpp_history if t - time <= 6]  # Filter last 6 seconds
    CPP_m = np.mean(past_cpp) if past_cpp else CPP
    CVR = CVR_m(CPP_m)
    Risf = Risf_span/CVR + Risf_offset
    Ca = Caspan/(CVR/CVR_min)+Coffset


    # ODEs from the given equations
    dPi_dt = (1 / Ci) * ((ABP - Pa) / Ra - (Pv - Pss) / Rb - (Pi - Pss) / RCSF - Pi/Risf) # #Node C
    dPa_dt = dPi_dt + (1 / Ca) * ((ABP - Pa) / Ra - (Pa - Pv) / CVR - If-I_inf) # Node A
    dPv_dt = dPi_dt + (1 / Cv) * ((Pa - Pv) / CVR - (Pv - Pss) / Rb) # Node B
    return [dPa_dt, dPv_dt, dPi_dt]

# Time span
t_span = (0, total_time)
t_eval = (np.arange(0, total_time, 0.04))
# Solve the ODE system
solution = solve_ivp(csf_dynamics, t_span, y0)


KeyboardInterrupt: 

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))

# Pressure Evolution Plot
plt.plot(solution.t, solution.y[0], label="Pa (Arterial Pressure)", color='r')
plt.plot(solution.t, solution.y[1], label="Pv (Venous Pressure)", color='b')
plt.plot(solution.t, solution.y[2], label="Pi (Intracranial Pressure)", color='g')
plt.xlabel("Time (s)")
plt.ylabel("Pressure (mmHg)")
plt.title("CSF Dynamics: Pressure Evolution Over Time")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
# CPP vs. CVR plot
CPP_m = np.linspace(0,200,201)
CVR = CVR_m(CPP_m)
plt.figure(figsize=(10, 6))
plt.plot(CPP_m,CVR)
plt.show()

In [ ]:
# ABP history plot
plt.figure(figsize=(10, 6))
times, abp_values = zip(*ABP_history)
plt.plot(times, abp_values, label="ABP (Arterial Blood Pressure)", color='orange')
plt.title("ABP Over Time")
plt.xlabel("Time [s]")
plt.ylabel("ABP [mmHg]")
plt.show()

In [ ]:

# CBF Calculation
plt.figure(figsize=(10, 6))
abp_interp_func = interp1d(times, abp_values, kind='linear', bounds_error=False, fill_value="extrapolate")
abp_resampled = abp_interp_func(solution.t)
CBF = (abp_resampled - solution.y[0]) / Ra
plt.plot(solution.t, CBF, label="CBF", color='orange')
plt.title("Cerebral Blood Flow Over Time")
plt.xlabel("Time (s)")
plt.ylabel("CBF")
plt.show()